In [ ]:
from appgeopy import *
from my_packages import *
from utils import *

#### GWR model

In [ ]:
df = pd.read_pickle("20250416_GWR_InputData_CUMDISP_MLCW_InSAR.xz")
cache = {}

layers = [col for col in df.columns if col.startswith("Layer")]
x_coord = "X_TWD97"
y_coord = "Y_TWD97"

indep_vars = ["CUMDISP"]
time_arr = df["time"]
unique_time = time_arr.unique()

# select_time = unique_time[0]
for select_time in tqdm(unique_time):
    
    df_byTime = df.query("time==@select_time")
    
    time2str = select_time.strftime("%Y%m%d")
    
    cache["N" + time2str] = {}
    
    for lay in layers:
        gwr_result = runGWR(
            main_df=df_byTime,
            target_colname=lay,
            predictors_collist=indep_vars,
            x_colname=x_coord,
            y_colname=y_coord,
        )
    
        gwr_r2adj = gwr_result.adj_R2
        globalReg_result = globalReg(main_df=df_byTime, target_colname=lay, predictors_collist=indep_vars)
        ols_r2adj = globalReg_result.rsquared_adj
    
        cache["N" + time2str][lay] = {
            "GWR_r2adj": round(gwr_r2adj, 3),
            "OLS_r2adj": round(ols_r2adj, 3),
        }
json_io(path="GWR_model_results_CUMDISP_data.json", mode="write", data=cache)

#### GTWR model

In [ ]:
from mgtwr.kernel import GTWRKernel
from mgtwr.model import GTWR
from mgtwr.sel import SearchGTWRParameter

df = pd.read_pickle("20250416_GWR_InputData_MLCW_InSAR.xz")
available_stations = df["STATION"].unique()
df["monthly"] = df["time"].dt.to_period("M")
df["monthly"] = df["monthly"].sub(df["monthly"].iloc[0]).apply(lambda x: x.n)

layers = [col for col in df.columns if col.startswith("Layer")]

x_coord = "X_TWD97"
y_coord = "Y_TWD97"
time_col = "monthly"

# --------------------------------------------------------
target_colname = layers[0]
coords = df.loc[:, [x_coord, y_coord]]
time_arr = df[[time_col]]
target_var = df.loc[:, [target_colname]]
predictors = df.loc[:, ["CUMDISP"]]

# --------------------------------------------------------
# Step 2: Calibrate the GTWR Model
# --------------------------------------------------------
sel = SearchGTWRParameter(
    coords=coords, t=time_arr, X=predictors, y=target_var, kernel="gaussian", fixed=True, thread=8
)

spatial_bw, temporal_bw = sel.search(
    criterion="CV",
    bw_min=0,
    bw_max=10,
    # tau_min=np.min(time_arr),
    # tau_max=np.max(time_arr),
    tau_min=0,
    tau_max=5,
    tol=1e-12,
    bw_decimal=0,
    tau_decimal=0,
    max_iter=1_000,
    verbose=True,
    time_cost=True,
)

gtwr = GTWR(coords=coords, t=time_arr, X=predictors, y=target_var, bw=spatial_bw, tau=temporal_bw, kernel='gaussian', fixed=True, thread=8).fit()
print(gtwr.R2)

In [ ]:
gtwr.predict_value.shape

In [ ]:
spatial_bw, temporal_bw